# Test data loading

In [ ]:
from bokeh.io import output_notebook
output_notebook()

## Bokeh Demo

In [ ]:
from bokeh.palettes import HighContrast3
from bokeh.plotting import figure, show

fruits = ["Apples", "Pears", "Nectarines", "Plums", "Grapes", "Strawberries"]
years = ["2015", "2016", "2017"]

data = {"fruits": fruits, "2015": [2, 1, 4, 3, 2, 4], "2016": [5, 3, 4, 2, 4, 6], "2017": [3, 2, 4, 4, 5, 3]}

p = figure(x_range=fruits, height=250, title="Fruit Counts by Year", toolbar_location=None, tools="hover", tooltips="$name @fruits: @$name")

p.vbar_stack(years, x="fruits", width=0.9, color=HighContrast3, source=data, legend_label=years)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)

In [ ]:
import json

with open('data/cleaned_business.json', 'r', encoding='utf-8') as file:
    businesses = json.load(file)

print(businesses[0])
print(businesses[0]['name'])

postal_codes = []
for business in businesses:
    postal_codes.append(business['postal_code'])

print(postal_codes[0:10])

In [ ]:
from collections import Counter
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6


postal_codes.sort()
postal_code_counts = Counter(postal_codes)
codes = list(postal_code_counts.keys())
counts = list(postal_code_counts.values())
print(codes)

p = figure(x_range=codes, height=400, title="Distribution of Postal Codes", toolbar_location=None,tools="")

p.vbar(x=codes, top=counts, width=0.9, color="#CAB2D6")
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.axis_label = "Postal Codes"
p.yaxis.axis_label = "Frequency"
p.xaxis.major_label_orientation = 1.3 


show(p)

# Map

## Hex-Binning

In [ ]:
import pandas as pd
from bokeh.plotting import figure, show
from pyproj import Proj, Transformer, CRS
import xyzservices.providers as xyz
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.models.glyphs import Scatter
from bokeh.transform import linear_cmap
from bokeh.util.hex import hexbin

output_notebook()

# load data and extract relevant columns and sample
df = pd.read_csv("data/cleaned_businessV2.csv")
#df = df.sample(100)
df = df[["name", "latitude", "longitude"]]


# calculate web mercator coordinates
in_proj = CRS.from_epsg(4326)   # WGS84
out_proj = CRS.from_epsg(3857)  # Web Mercator
transformer = Transformer.from_crs(in_proj, out_proj, always_xy=True)
def latlon_to_web_mercator(lat, lon):
    x, y = transformer.transform(lon, lat)
    return x, y
df['x'], df['y'] = zip(*df.apply(lambda x: latlon_to_web_mercator(x.latitude, x.longitude), axis=1))
max_x = df['x'].max()
min_x = df['x'].min()
max_y = df['y'].max()
min_y = df['y'].min()

# create plot
source = ColumnDataSource(df)
p = figure(title="Philadelphia Restaurants",
           x_axis_type="mercator", y_axis_type="mercator",
           x_range=(min_x, max_x), y_range=(min_y, max_y),
           width=800, height=600,
           tools="wheel_zoom,pan,reset, box_select, lasso_select")
p.add_tile(xyz.CartoDB.Positron)

p.grid.visible = False

bins = hexbin(df['x'], df['y'], size=500)
p.hex_tile(q="q", r="r",source = bins,
           size=500, line_color=None, fill_alpha=0.5,
           fill_color=linear_cmap('counts', 'Viridis256', 0, max(bins.counts)))


glyph = Scatter(x='x', y='y', size=3, fill_color='black', fill_alpha=0.1, line_color=None)
p.add_glyph(source, glyph)

source.selected.js_on_change('indices', CustomJS(args=dict(source=source), code="""
    const indices = cb_obj.indices;
    console.log('Number of selected points:', indices.length);
    """))

show(p)


## Kernel Density Estimation

In [ ]:
import pandas as pd
from bokeh.plotting import figure, show
from pyproj import Transformer, CRS
import xyzservices.providers as xyz
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.models.glyphs import Scatter
from scipy.stats import gaussian_kde
import numpy as np
from bokeh.palettes import Viridis256

output_notebook()

# load data and extract relevant columns and sample
df = pd.read_csv("data/cleaned_businessV2.csv")
#df = df.sample(100)
df = df[df['categories'].str.contains("Korean")]
df.head()
df = df[["name", "latitude", "longitude"]]


# calculate web mercator coordinates
in_proj = CRS.from_epsg(4326)   # WGS84
out_proj = CRS.from_epsg(3857)  # Web Mercator
transformer = Transformer.from_crs(in_proj, out_proj, always_xy=True)
def latlon_to_web_mercator(lat, lon):
    x, y = transformer.transform(lon, lat)
    return x, y
df['x'], df['y'] = zip(*df.apply(lambda x: latlon_to_web_mercator(x.latitude, x.longitude), axis=1))
max_x, min_x = df['x'].max(), df['x'].min()
max_y, min_y = df['y'].max(), df['y'].min()

# create plot
source = ColumnDataSource(df)
p = figure(title="Philadelphia Restaurants",
           x_axis_type="mercator", y_axis_type="mercator",
           x_range=(min_x, max_x), y_range=(min_y, max_y),
           width=800, height=600,
           tools="wheel_zoom, pan, reset, box_select, lasso_select")
p.add_tile(xyz.CartoDB.Positron)
p.grid.visible = False

# add kernel density estimation for heatmap
def kde(x,y,N):
    X, Y = np.mgrid[min_x:max_x:N*1j, min_y:max_y:N*1j]
    positions = np.vstack([X.ravel(), Y.ravel()])
    values = np.vstack([x, y])
    kernel = gaussian_kde(values)
    Z = np.reshape(kernel(positions).T, X.shape)
    return X, Y, Z

density_x, density_y, density = kde(df['x'], df['y'], 100)
palette = Viridis256[::-1]
levels = np.linspace(np.min(density), np.max(density), len(palette))

p.contour(density_x, density_y, density, levels=levels,
          fill_color=palette, line_color=None, fill_alpha=0.3)

# add points
glyph = Scatter(x='x', y='y', size=3, fill_color='black', fill_alpha=0.1, line_color=None)
p.add_glyph(source, glyph)

show(p)


## KDE With transparency

In [ ]:
import pandas as pd
from bokeh.plotting import figure, show
from pyproj import Transformer, CRS
import xyzservices.providers as xyz
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.models.glyphs import Scatter
from scipy.stats import gaussian_kde
import numpy as np
from bokeh.palettes import viridis, magma

output_notebook()

# load data and extract relevant columns and sample
df = pd.read_csv("data/cleaned_businessV2.csv")
#df = df.sample(100)
#df = df[df['categories'].str.contains("Korean")]
df.head()
df = df[["name", "latitude", "longitude"]]


# calculate web mercator coordinates
in_proj = CRS.from_epsg(4326)   # WGS84
out_proj = CRS.from_epsg(3857)  # Web Mercator
transformer = Transformer.from_crs(in_proj, out_proj, always_xy=True)
def latlon_to_web_mercator(lat, lon):
    x, y = transformer.transform(lon, lat)
    return x, y
df['x'], df['y'] = zip(*df.apply(lambda x: latlon_to_web_mercator(x.latitude, x.longitude), axis=1))
max_x, min_x = df['x'].max(), df['x'].min()
max_y, min_y = df['y'].max(), df['y'].min()

# create plot
buffer = 1.2
source = ColumnDataSource(df)
p = figure(title="Philadelphia Restaurants",
           x_axis_type="mercator", y_axis_type="mercator",
           x_range=(min_x, max_x), y_range=(min_y, max_y),
           width=800, height=600,
           tools="wheel_zoom, pan, reset, box_select, lasso_select")
p.add_tile(xyz.CartoDB.Positron)
p.grid.visible = False

# add kernel density estimation for heatmap
def kde(x,y,N):
    X, Y = np.mgrid[min_x:max_x:N*1j, min_y:max_y:N*1j]
    positions = np.vstack([X.ravel(), Y.ravel()])
    values = np.vstack([x, y])
    kernel = gaussian_kde(values)
    Z = np.reshape(kernel(positions).T, X.shape)
    return X, Y, Z

density_x, density_y, density = kde(df['x'], df['y'], 100)

palette = list(viridis(40))[::-1]
palette.insert(0, "#00000000")
levels = np.linspace(np.min(density), np.max(density), len(palette))

p.contour(density_x, density_y, density, levels=levels,
          fill_color=palette, line_color=None, fill_alpha=0.3)

# add points
glyph = Scatter(x='x', y='y', size=3, fill_color='black', fill_alpha=0.1, line_color=None, line_width=0)
p.add_glyph(source, glyph)

show(p)
